In [1]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import ast
from models.ActFrameConstructor import *

# TODO: there must be some cleaner way use local packages with poetry, but lets use this quick hack for now...
import sys
sys.path.append('../../')  # Adjust the path as needed
from utils.spacy_visualizer import SpacyLabelVisualizer


In [2]:
load_dotenv(find_dotenv())
OPENAI_SECRET_KEY = os.getenv('OPENAI_SECRET_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_SECRET_KEY

# Configuration

In [3]:
prompt_file_name = 'prompt_3.txt'
gpt_model = Model.gpt35turbo
article_name = "GDPR_6" #TODO: make enum class for articles

In [4]:
prompt1 = open(f'system_prompts/{prompt_file_name}', 'r').read()

In [5]:
predictor = ActFrameConstructor(prompt=prompt1, model=gpt_model, prompt_id=prompt_file_name)

In [6]:
article = open(f'data/{article_name}.txt', 'r').read()

In [7]:
answer = predictor.request_full_response(article)

In [8]:
# save answer in a file
answer_id  = predictor.id() + f":article:{article_name}" + f":date:{pd.Timestamp.now()}"
with open(f'answers/{answer_id}.md', 'w') as f: # write as markdown; as the reponse will contain a json block this will allow for an easier read.
    f.write(answer)

In [9]:
# TODO: use another pormpt with json mode and extract answer
# TODO: give room to think (COT); given the answers right now, it seems to jump straight to the answer instead.